In [1]:
import sys
from pathlib import Path

# Add src directory to Python path
project_root = Path.cwd().parent  
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to Python path")


Added /Users/cooper/Desktop/hydro-forecasting/src to Python path


In [2]:
import polars as pl
from hydro_forecasting.data.index_entry_creator import find_valid_sequences

---

In [5]:
data = pl.read_parquet("/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/5af0c3ea-c280-59e5-83f0-bcf41247b3ab/processed_time_series/test/USA_01030500.parquet")

# Drop gauge_id column
data = data.drop("gauge_id")
data

date,snow_depth_water_equivalent_mean,surface_net_solar_radiation_mean,surface_net_thermal_radiation_mean,potential_evaporation_sum_ERA5_LAND,potential_evaporation_sum_FAO_PENMAN_MONTEITH,temperature_2m_mean,temperature_2m_min,temperature_2m_max,total_precipitation_sum,streamflow
datetime[ns],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2006-01-26 00:00:00,0.105601,-0.844563,-0.344775,-0.279859,-0.760783,-1.044429,-0.974075,-1.217039,-0.491804,1.284559
2006-01-27 00:00:00,0.093413,-0.684464,-0.720113,-0.213466,-0.715711,-1.103484,-1.077213,-1.09882,-0.499845,0.91973
2006-01-28 00:00:00,0.077709,-0.683697,-0.29569,-0.192656,-0.657761,-0.370151,-0.490453,-0.399862,-0.499845,0.614186
2006-01-29 00:00:00,0.062942,-0.723051,0.185938,-0.173829,-0.638445,-0.729766,-0.616125,-0.808882,-0.32616,0.445452
2006-01-30 00:00:00,0.135838,-1.128346,1.651491,-0.749561,-0.760783,-0.963339,-0.754798,-1.175619,0.035684,0.272159
…,…,…,…,…,…,…,…,…,…,…
2014-10-01 00:00:00,-0.575073,-0.773266,1.385765,-0.383907,-0.348696,0.549159,0.776673,0.370717,-0.485371,-0.763043
2014-10-02 00:00:00,-0.575073,0.521071,-0.953731,0.379112,0.17929,0.477765,0.514928,0.523452,-0.498236,-0.763043
2014-10-03 00:00:00,-0.575073,0.344234,-0.481329,0.054086,0.089146,0.498919,0.446458,0.57609,-0.498236,-0.763043


In [6]:
positions, dates = find_valid_sequences(data, 10, 10)

In [9]:
positions

array([   0,    1,    2, ..., 3153, 3154, 3155], shape=(3156,))

In [8]:
dates

array(['2006-01-26T00:00:00.000000000', '2006-01-27T00:00:00.000000000',
       '2006-01-28T00:00:00.000000000', ...,
       '2014-10-03T00:00:00.000000000', '2014-10-04T00:00:00.000000000',
       '2014-10-05T00:00:00.000000000'],
      shape=(3175,), dtype='datetime64[ns]')